In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:
# Importing the libraries:
import tensorflow as tf
from tensorflow import keras
import numpy as np
tf.__version__, keras.__version__

('2.10.0', '2.10.0')

# Prepping the Images:

In [3]:
# Using a function to automate image loading and transformations:
path = "images/"

data = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    label_mode = None, # To ensure that it doesn't throw me an error and retrieves all the images.
    color_mode = "rgb",
    batch_size = 128,
    image_size = (100, 100),
    shuffle = True
)
data = data.unbatch()
data = data.batch(128, drop_remainder = True)

Found 63565 files belonging to 1 classes.


In [4]:
# Creating some more variables:
coding_size = 128
optimizer = tf.keras.optimizers.Nadam(learning_rate = 0.0002, beta_1 = 0.5)
loss = tf.keras.losses.binary_crossentropy
batch_size = 128

# Creating a GAN:

In [5]:
# Sequential generator:
generator = keras.models.Sequential([
    keras.layers.Dense(5 * 5 * 1024, input_shape = [coding_size]),
    keras.layers.Reshape([5, 5, 1024]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size = 5, strides = 2, padding = "same", activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(128, kernel_size = 5, strides = 5, padding = "same", activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(3, kernel_size = 5, strides = 2, padding = "same", activation = "tanh")
])

In [6]:
# Sequential discriminator:
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size = 5, strides  = 2, padding = "same", activation = tf.keras.layers.LeakyReLU(0.2), input_shape = [100, 100, 3]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size = 5, strides = 2, padding = "same", activation = tf.keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(256, kernel_size = 5, strides  = 2, padding = "same", activation = tf.keras.layers.LeakyReLU(0.2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation = "sigmoid")
])

In [7]:
# Creating a gan:
gan = keras.models.Sequential([generator, discriminator])

# Training the GAN:

In [8]:
# Creating a image plotter:
def plot(x_train, image_no = 1):
    for i in tf.range(image_no):
        img = x_train[i]
        img = tf.keras.preprocessing.image.array_to_img(img)
        img.show()


In [9]:
# Creating an custom callback:
def Checkpoint(model):
    model.save("Checkpoints/GAN.h5")

In [10]:
# General Step:
def train_step(model, x, y, loss_fn = loss, optimizer = optimizer):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y_pred = y_pred, y_true = y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [11]:
#  Discriminator step:
@tf.function
def train_d(model, noise, x_batch, batch_size = batch_size):
    generator, discriminator = model.layers
    predictions = generator(noise)
    inputs = tf.concat([predictions, x_batch], axis = 0)
    y = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
    discriminator.trainable = True
    train_step(discriminator, inputs, y)

In [12]:
# Generator step:
@tf.function
def train_g(model, noise, x_batch, batch_size = batch_size):
    discriminator = model.layers[1]
    discriminator.trainable = False
    y = tf.constant([[1]] * batch_size)
    train_step(model, noise, y)

In [13]:
# Predicting values:
def predict(model, batch_size = batch_size, coding_size = coding_size):
    noise = tf.random.normal([batch_size, coding_size])
    generator = model.layers[0]
    predictions = generator(noise)
    plot(predictions)

In [14]:
# Training Loop:
epochs = 5
for epoch in range(epochs):
    for x_batch in data:
        # Training the Discriminator:
        noise = tf.random.normal([batch_size, coding_size])
        train_d(gan, noise, x_batch)
        # Training the Generator:
        noise = tf.random.normal([batch_size, coding_size])
        train_g(gan, noise, x_batch)
    print(f"Epochs {epoch + 1} passed.")
    predict(gan)
    Checkpoint(gan)

Epochs 1 passed.


Epochs 2 passed.


Epochs 3 passed.



(eog:9497): Gtk-WARNING **: 07:22:27.066: Negative content width -1 (allocation 23, extents 12x12) while allocating gadget (node label, owner GtkLabel)
